In [118]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [119]:
import logging
import sys

from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.feature_selection import f_classif, chi2, SelectPercentile, RFECV, SelectKBest
from sklearn.metrics import f1_score
from imblearn.metrics import classification_report_imbalanced

from lightgbm import LGBMClassifier as LGBM
from catboost import CatBoostClassifier as CatBoost
from xgboost import XGBClassifier as XGB
from sklearn.preprocessing import RobustScaler, Normalizer, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier as RandomForest, AdaBoostClassifier as AdaBoost, 
                              HistGradientBoostingClassifier as HistGradBoost, StackingClassifier, VotingClassifier)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [120]:
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

In [121]:
%%time
df_train = pd.read_csv("train_tfidf_features.csv")
display(df_train)

,id,label,0,1,2,3,4,5,6,7,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,17180,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17180,17181,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17181,17182,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17182,17183,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wall time: 18.2 s


In [122]:
# df_train.iloc[:,1:8:2].boxplot(by="label")

In [123]:
X = df_train.iloc[:, 2:5002].to_numpy()
y = df_train.iloc[:,1].to_numpy()
print(f"X: {X.shape}, y: {y.shape}")

X: (17184, 5000), y: (17184,)


In [124]:
# df_test = pd.read_csv("./test_tfidf_features.csv")
# display(df_test)

In [125]:
# X_test = df_test.iloc[:, 1:5001].to_numpy()
# helper = df_test.iloc[:, 0]
# display(X_test)
# display(helper)

In [126]:
# scaler = RobustScaler(with_centering=False, unit_variance=True).fit(X_train)
# scaler = Normalizer(norm='max', copy=True).fit(X_train)
# scaler = MinMaxScaler(feature_range=(0, 1), copy=True, clip=False).fit(X_train)
# scaler = MaxAbsScaler(copy=True).fit(X_train)
# X_trans = scaler.transform(X_train)
# X_train = kernel.fit_transform(X_train)

X_new = SelectPercentile(chi2, percentile = 50).fit_transform(X, y)

X_train, X_sub, y_train, y_sub = train_test_split(X_new, y, test_size=0.25, random_state=100)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_valid: {X_sub.shape}, y_valid: {y_sub.shape}")

X_train: (12888, 2500), y_train: (12888,)
X_valid: (4296, 2500), y_valid: (4296,)


In [127]:
# scaler = Normalizer(norm='max', copy=True).fit(X_train)
# X_trans = scaler.transform(X_train)
# norm = pd.DataFrame(X_trans) # Normalizer
# display(norm.describe())
# norm.var().sort_values(ascending=False).head(20)

In [128]:
# scaler = RobustScaler(with_centering=False, unit_variance=True).fit(X_train)
# X_trans = scaler.transform(X_train)
# robust = pd.DataFrame(X_trans) # RobustScaler
# display(robust.describe())
# robust.var().sort_values(ascending=False).head(20)

In [129]:
# scaler = MinMaxScaler(feature_range=(0, 1), copy=True, clip=False).fit(X_train)
# X_trans = scaler.transform(X_train)
# minmax = pd.DataFrame(X_trans) # MinMaxScaler
# display(minmax.describe())
# minmax.var().sort_values(ascending=False).head(20)

In [130]:
# scaler = MaxAbsScaler(copy=True).fit(X_train)
# X_trans = scaler.transform(X_train)
# maxabs = pd.DataFrame(X_trans) # MaxAbsScaler
# display(maxabs.describe())
# maxabs.var().sort_values(ascending=False).head(20)

In [131]:
original = pd.DataFrame(X_train) # original
display(original.describe())
original.var().sort_values(ascending=False).head(20)

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
count,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,...,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000
mean,0.000963,0.000453,0.000463,0.000148,0.000767,0.000185,0.000189,0.000216,0.000213,0.000097,...,0.000191,0.000510,0.000604,0.000164,0.000230,0.000071,0.000229,0.000774,0.000272,0.000264
std,0.018665,0.012696,0.014231,0.007533,0.016931,0.008634,0.009901,0.011760,0.010924,0.006336,...,0.009882,0.013836,0.015301,0.008560,0.010936,0.005948,0.009854,0.018731,0.010389,0.009601
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.560830,0.536746,0.550237,0.422529,0.689683,0.477186,0.614334,0.978105,0.628086,0.434397,...,0.653012,0.551582,0.532189,0.570588,0.611122,0.592170,0.469045,0.850605,0.484887,0.427472


2434    0.009094
1635    0.006384
1788    0.005776
1020    0.004696
262     0.004413
1211    0.003690
2182    0.003411
1785    0.003351
2375    0.003191
1247    0.003029
1113    0.002973
1315    0.002878
674     0.002865
132     0.002732
1900    0.002674
117     0.002618
1472    0.002424
2312    0.002345
1360    0.002300
2412    0.002282
dtype: float64

In [132]:
X_valid, X_test, y_valid, y_test = train_test_split(X_sub, y_sub, test_size=0.5,random_state=200)
print(f"X_valid: {X_valid.shape}, y_valid: {y_valid.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_valid: (2148, 2500), y_valid: (2148,)
X_test: (2148, 2500), y_test: (2148,)


In [133]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
count,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,...,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000,12888.000000
mean,0.000963,0.000453,0.000463,0.000148,0.000767,0.000185,0.000189,0.000216,0.000213,0.000097,...,0.000191,0.000510,0.000604,0.000164,0.000230,0.000071,0.000229,0.000774,0.000272,0.000264
std,0.018665,0.012696,0.014231,0.007533,0.016931,0.008634,0.009901,0.011760,0.010924,0.006336,...,0.009882,0.013836,0.015301,0.008560,0.010936,0.005948,0.009854,0.018731,0.010389,0.009601
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.560830,0.536746,0.550237,0.422529,0.689683,0.477186,0.614334,0.978105,0.628086,0.434397,...,0.653012,0.551582,0.532189,0.570588,0.611122,0.592170,0.469045,0.850605,0.484887,0.427472


In [134]:
pd.DataFrame(X_valid).describe()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
count,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.0,2148.0,2148.000000,2148.0,2148.000000,...,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.0,2148.000000,2148.000000,2148.0,2148.000000
mean,0.001918,0.000163,0.000229,0.000201,0.000465,0.0,0.0,0.000184,0.0,0.000202,...,0.000213,0.000564,0.000669,0.000135,0.000234,0.0,0.000702,0.000344,0.0,0.000876
std,0.026637,0.007532,0.007526,0.009298,0.012861,0.0,0.0,0.008517,0.0,0.009370,...,0.009850,0.015393,0.020784,0.006261,0.010825,0.0,0.019013,0.011325,0.0,0.018248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
max,0.547623,0.349101,0.256992,0.430946,0.446216,0.0,0.0,0.394740,0.0,0.434273,...,0.456505,0.509092,0.877622,0.290194,0.501720,0.0,0.617341,0.406691,0.0,0.430031


In [135]:
pd.DataFrame(X_test).describe()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
count,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.0,2148.0,2148.000000,...,2148.0,2148.000000,2148.000000,2148.0,2148.0,2148.000000,2148.0,2148.000000,2148.000000,2148.0
mean,0.000830,0.000985,0.000338,0.000184,0.000827,0.000717,0.000166,0.0,0.0,0.000395,...,0.0,0.000524,0.000274,0.0,0.0,0.000171,0.0,0.000687,0.000422,0.0
std,0.015868,0.018710,0.011156,0.008530,0.018151,0.020574,0.007678,0.0,0.0,0.012984,...,0.0,0.014283,0.009206,0.0,0.0,0.007942,0.0,0.016013,0.013862,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
max,0.381405,0.417587,0.408241,0.395327,0.600363,0.794519,0.355842,0.0,0.0,0.456960,...,0.0,0.473135,0.362805,0.0,0.0,0.368102,0.0,0.440547,0.484908,0.0


In [136]:
pd.DataFrame(y_train).describe()

,0
count,12888.000000
mean,0.383457
std,0.486247
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [137]:
pd.DataFrame(y_valid).describe()

,0
count,2148.000000
mean,0.374767
std,0.484175
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [138]:
pd.DataFrame(y_test).describe()

,0
count,2148.000000
mean,0.374302
std,0.484055
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [139]:
def macro_f1_score(y, y_hat):
    accuracy = np.sum(y == y_hat) / np.shape(y)[0]
    score = f1_score(y, y_hat, average='macro')

    print(f"Accuracy: {accuracy}")
    print(f"Macro-F1 score: {score}")

    # Return Macro-F1 score of the model
    return score

In [140]:
def create_model(trial):
    params = {
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart','goss']), #added goss
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 80, 200),
        'num_leaves' : trial.suggest_int('num_leaves', 100, 250),
        
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.3, 0.8),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.25, 2), # added scale_pos_weight
        
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20, step=1),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-8, 0.2, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 1e-4, 0.3, log=True),
        
        'max_delta_step': trial.suggest_float('max_delta_step', 0.1, 1.75),
        'n_estimators': trial.suggest_int('n_estimators', 20, 120, step=5),
        
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 100.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 100.0, log=True),
        
        'random_state': 100,
        'n_jobs': -1
    }
    
    model = LGBM(**params)
    
    return model

In [141]:
# from sklearn.manifold import MDS, Isomap, SpectralEmbedding, TSNE, LocallyLinearEmbedding

In [142]:
# def objective(trial, X_train=X_train, y_train=y_train,X_valid=X_valid, y_valid=y_valid,model=create_model): 
#     model = create_model(trial) 
#     model.fit(X_train, y_train)
#     preds = model.predict(X_valid)
#     score = macro_f1_score(y_valid, preds)
#     return score

In [143]:
# X_red_train = X_train
# X_red_valid = X_valid
# X_red_test = X_test

In [149]:
svd = TruncatedSVD(n_components=int(X_train.shape[1]*0.6), random_state=100, n_iter=8)
svd.fit(X_train, y_train)
X_svm_train = svd.transform(X_train)
X_svm_valid = svd.transform(X_valid)
X_svm_test = svd.transform(X_test)

In [150]:
# X_final_train = X_red_train
# X_final_valid = X_red_valid
# X_final_test = X_red_test

In [151]:
# ‘additive_chi2’, ‘chi2’, ‘linear’, ‘rbf’, ‘laplacian’, ‘sigmoid’, ‘cosine
# kernel = Nystroem(kernel='cosine', gamma=0.65, coef0=0, n_components=2000, random_state=100, n_jobs=-1)
# kernel.fit(X_train)
# X_train = kernel.transform(X_train)
# X_valid = kernel.transform(X_valid)
# X_test = kernel.transform(X_test)

scaler = Normalizer(norm='max', copy=True).fit(X_svm_train)
# scaler = RobustScaler(with_centering=False, unit_variance=True).fit(X_red_train)
# scaler = MinMaxScaler(feature_range=(0, 1), copy=True, clip=False).fit(X_red_train)
# scaler = MaxAbsScaler(copy=True).fit(X_red_train)
X_final_train = scaler.transform(X_svm_train)
X_final_valid = scaler.transform(X_svm_valid)
X_final_test = scaler.transform(X_svm_test)

In [152]:
def objective(trial, X_train=X_final_train, y_train=y_train,X_valid=X_final_valid, y_valid=y_valid,model=create_model):

    model = create_model(trial)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = macro_f1_score(y_valid, preds)
    return score

In [153]:
%%time
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize') # can define seed for sampler
study.optimize(objective, n_trials=50)

[I 2022-08-02 13:44:12,328] A new study created in memory with name: no-name-7c424004-2932-45a2-a915-2f19db8e0557


A new study created in memory with name: no-name-7c424004-2932-45a2-a915-2f19db8e0557
A new study created in memory with name: no-name-7c424004-2932-45a2-a915-2f19db8e0557
A new study created in memory with name: no-name-7c424004-2932-45a2-a915-2f19db8e0557
A new study created in memory with name: no-name-7c424004-2932-45a2-a915-2f19db8e0557


[I 2022-08-02 13:44:17,867] Trial 0 finished with value: 0.6823121450415028 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.1301212381528452, 'max_depth': 117, 'num_leaves': 138, 'colsample_bytree': 0.668499891513124, 'subsample': 0.6959886441779408, 'scale_pos_weight': 1.601800119483772, 'min_child_samples': 17, 'min_child_weight': 3.581148532418961e-06, 'min_split_gain': 0.28263324353011804, 'max_delta_step': 1.5748395122191405, 'n_estimators': 20, 'reg_alpha': 64.79899595506276, 'reg_lambda': 0.06417344409538477}. Best is trial 0 with value: 0.6823121450415028.


Accuracy: 0.6973929236499069
Macro-F1 score: 0.6823121450415028
Trial 0 finished with value: 0.6823121450415028 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.1301212381528452, 'max_depth': 117, 'num_leaves': 138, 'colsample_bytree': 0.668499891513124, 'subsample': 0.6959886441779408, 'scale_pos_weight': 1.601800119483772, 'min_child_samples': 17, 'min_child_weight': 3.581148532418961e-06, 'min_split_gain': 0.28263324353011804, 'max_delta_step': 1.5748395122191405, 'n_estimators': 20, 'reg_alpha': 64.79899595506276, 'reg_lambda': 0.06417344409538477}. Best is trial 0 with value: 0.6823121450415028.
Trial 0 finished with value: 0.6823121450415028 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.1301212381528452, 'max_depth': 117, 'num_leaves': 138, 'colsample_bytree': 0.668499891513124, 'subsample': 0.6959886441779408, 'scale_pos_weight': 1.601800119483772, 'min_child_samples': 17, 'min_child_weight': 3.581148532418961e-06, 'min_split_gain': 0.28263324353011804

[I 2022-08-02 13:44:26,139] Trial 1 finished with value: 0.5547030317006104 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.028917517501997947, 'max_depth': 185, 'num_leaves': 216, 'colsample_bytree': 0.5625886764275239, 'subsample': 0.6418075782836599, 'scale_pos_weight': 1.469399092351092, 'min_child_samples': 20, 'min_child_weight': 0.052664519909166846, 'min_split_gain': 0.20470021551960166, 'max_delta_step': 0.43977197498737697, 'n_estimators': 45, 'reg_alpha': 21.02454989902164, 'reg_lambda': 0.07458033049010115}. Best is trial 0 with value: 0.6823121450415028.


...

[I 2022-08-02 14:00:11,371] Trial 22 finished with value: 0.7088203397999759 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.05847869157449784, 'max_depth': 131, 'num_leaves': 135, 'colsample_bytree': 0.9379398177670066, 'subsample': 0.4263951167427954, 'scale_pos_weight': 1.5157762241665758, 'min_child_samples': 3, 'min_child_weight': 0.0007125536049525366, 'min_split_gain': 0.003963091165284963, 'max_delta_step': 1.6419544202642156, 'n_estimators': 80, 'reg_alpha': 0.024930697494265616, 'reg_lambda': 0.017053677668765616}. Best is trial 18 with value: 0.7103310076068154.


Accuracy: 0.7318435754189944
Macro-F1 score: 0.7088203397999759
Trial 22 finished with value: 0.7088203397999759 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.05847869157449784, 'max_depth': 131, 'num_leaves': 135, 'colsample_bytree': 0.9379398177670066, 'subsample': 0.4263951167427954, 'scale_pos_weight': 1.5157762241665758, 'min_child_samples': 3, 'min_child_weight': 0.0007125536049525366, 'min_split_gain': 0.003963091165284963, 'max_delta_step': 1.6419544202642156, 'n_estimators': 80, 'reg_alpha': 0.024930697494265616, 'reg_lambda': 0.017053677668765616}. Best is trial 18 with value: 0.7103310076068154.
Trial 22 finished with value: 0.7088203397999759 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.05847869157449784, 'max_depth': 131, 'num_leaves': 135, 'colsample_bytree': 0.9379398177670066, 'subsample': 0.4263951167427954, 'scale_pos_weight': 1.5157762241665758, 'min_child_samples': 3, 'min_child_weight': 0.0007125536049525366, 'min_split_gain': 0.00396

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\LENOVO PC\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\execution.py", line 1329, in time
    out = eval(code_2, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "C:\Users\LENOVO PC\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\study.py", line 400, in optimize
    _optimize(
  File "C:\Users\LENOVO PC\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 66, in _optimize
    _optimize_sequential(
  File "C:\Users\LENOVO PC\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 163, in _optimize_sequential
    trial = _run_trial(study, func, catch)
  File "C:\Users\LENOVO PC\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\LENOVO~1\AppData\Local\Temp/ipykernel_9932/3110114280.py", 

TypeError: object of type 'NoneType' has no len()

In [ ]:
print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial)
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_trial.value)  # Show the best value.
# Number of finished trials: 100
# Best parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.4561329620625593, 'max_depth': 177, 'num_leaves': 247, 'colsample_bytree': 0.8047370664395848, 'subsample': 0.6379228123349399, 'min_child_samples': 3, 'min_child_weight': 0.06313731736777115, 'min_split_gain': 0.00018796593923741422, 'max_delta_step': 0.11425890485056074, 'n_estimators': 65, 'reg_alpha': 0.024282143648941008, 'reg_lambda': 36.62235751478392}
# Best score: 0.7086699840268176
#  Trial 22 finished with value: 0.7100998639595939 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.35374751233313123, \
# 'max_depth': 149, 'num_leaves': 204, 'colsample_bytree': 0.5499055719152047, 'subsample': 0.5683393077019018, 'min_child_samples': 2, \
# 'min_child_weight': 0.09068141061818547, 'min_split_gain': 0.23576238602317104, 'max_delta_step': 0.1226383774396453, \
# 'n_estimators': 120, 'reg_alpha': 0.0009793960104005541, 'reg_lambda': 0.0002700644417464759}. Best is trial 22 with value: 0.7100998639595939.

In [ ]:
# model = LGBM(boosting_type='gbdt', learning_rate=0.21, max_depth=110, num_leaves=138, colsample_bytree=0.905, subsample=0.44, min_child_samples=6, min_child_weight=5, \
#              min_split_gain=0.007, max_delta_step=0.651, n_estimators=76, reg_alpha=0.205, reg_lambda=1.70, random_state=100, n_jobs=-1)
model = LGBM(**study.best_trial.params)
model.fit(X_final_train, y_train)
preds = model.predict(X_final_test)
score = macro_f1_score(y_test, preds)

In [ ]:
print(classification_report_imbalanced(y_test, preds))

In [ ]:
# df_trials = study.trials_dataframe()

In [ ]:
# df_trials.to_csv("LGBM_Hyperparams.csv", index=False)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['num_leaves',
                            'max_depth',
                            'subsample',
                            'learning_rate'])

In [ ]:
def create_model(trial):
    params = {
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain']),# 'Ordered', 
        'loss_function': 'Logloss',

        'learning_rate': trial.suggest_float('learning_rate', 0.05, 1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
#         'num_leaves' : trial.suggest_int('num_leaves', 80, 200),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Lossguide', 'Depthwise']), # 'Region' Unimplemented for CPU
        
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
#         'subsample': trial.suggest_float('subsample', 0.3, 0.8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 60, step=1),
        
        'n_estimators': trial.suggest_int('n_estimators', 60, 120, step=5),
        
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.25, 2.5), # instead of auto class weights
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 25, log=True),
        
        'random_state': 100,
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'eval_metric': 'F1', 
        'custom_metric': ['F1', 'Precision', 'Recall'],
    }

    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)
    elif params['bootstrap_type'] == 'MVS':
        params['mvs_reg'] = trial.suggest_float('mvs_reg', 0.5, 10)
    
    model = CatBoost(**params)
    
    return model

In [ ]:
def objective(trial, X_train=X_final_train, y_train=y_train,X_valid=X_final_valid, y_valid=y_valid,model=create_model):
    model = create_model(trial)
    model.fit(X_train, 
              y_train,
              eval_set=[(X_valid, y_valid)],
              verbose=0,
#               early_stopping_rounds=25
             )
    
    preds = model.predict(X_valid)
    score = macro_f1_score(y_valid, preds)
    
    return score

In [ ]:
%%time
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize') # can define seed for sampler
study.optimize(objective, n_trials=60, timeout=10000)

In [ ]:
print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial)
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_trial.value)  # Show the best value.
# Trial 153 finished with value: 0.7130286499403973 and parameters: {'boosting_type': 'Plain', 'learning_rate': 0.2417214498236795, 
# 'max_depth': 15, 'grow_policy': 'Depthwise', 'colsample_bytree': 0.5948706454819891, 'min_child_samples': 17, 'n_estimators': 90, 
# 'auto_class_weights': 'Balanced', 'reg_lambda': 5.267221485280441, 'bootstrap_type': 'MVS', 'mvs_reg': 5.555855456645176}

# Best parameters: {'boosting_type': 'Plain', 'learning_rate': 0.2510275559464182, 
#'max_depth': 15,'grow_policy': 'Depthwise', 'colsample_bytree': 0.42720150233770604, 'min_child_samples': 20, 
#'n_estimators': 95, 'auto_class_weights': 'Balanced', 'reg_lambda': 10.96439478226534, 
#'bootstrap_type': 'Bernoulli', 'subsample': 0.5333795300921985}
# Best score: 0.7087476093663909

In [ ]:
model = CatBoost(**study.best_trial.params)
model.fit(X_final_train, y_train)
preds = model.predict(X_final_test)
score = macro_f1_score(y_test, preds)

In [ ]:
df_trials = study.trials_dataframe()

In [ ]:
df_trials.to_csv("Catboost_Hyperparams.csv", index=False)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['scale_pos_weight',
                            'colsample_bylevel',
                            'subsample',
                            'learning_rate'])

In [ ]:
def create_model(trial):
    params = {
        'penalty': trial.suggest_categorical('penalty', ['l2', 'none']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sag', 'saga']),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),

        'max_iter': trial.suggest_int('max_iter', 300, 400),

        'verbose': 0,
        'random_state': 100,
        'n_jobs': -1,
    }
    
    if params['penalty'] == 'l2':
        params['C'] = trial.suggest_float('C', 0.1, 10, log=True)
        
    model = LogisticRegression(**params)
    
    return model

In [ ]:
def objective(trial, X_train=X_final_train, y_train=y_train,X_valid=X_final_valid, y_valid=y_valid,model=create_model):
#     svd_params = {
#         'n_components': trial.suggest_int('n_components', 50, 3500, step=50),
#         'algorithm': 'randomized', #trial.suggest_categorical('algorithm', ['arpack', 'randomized']),
#         'random_state': 100,
#         'n_iter': 5
#     }
    
#     svd = TruncatedSVD(**svd_params)
#     svd.fit(X_train)
#     X_train_svd = svd.transform(X_train)
    
    model = create_model(trial)
    
#     model.fit(X_train_svd, y_train)
#     X_valid_svd = svd.transform(X_valid)
#     preds = model.predict(X_valid_svd)

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = macro_f1_score(y_valid, preds)
    
    return score

In [ ]:
%%time
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize') # can define seed for sampler
study.optimize(objective, n_trials=60, timeout=10000)

In [ ]:
print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial)
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_trial.value)  # Show the best value.
# Number of finished trials: 5
# Best parameters: {'n_components': 2300, 'penalty': 'l2', 'solver': 'saga', 'class_weight': 'balanced', 'C': 0.740083647626061}
# Best score: 0.7122960641591057
# Number of finished trials: 60
# Best parameters: {'n_components': 3150, 'penalty': 'l2', 'solver': 'saga', 'class_weight': 'balanced', 'max_iter': 206}
# Best score: 0.7156894279571357
# Number of finished trials: 60
# Best parameters: {'n_components': 2950, 'penalty': 'l2', 'solver': 'lbfgs', 'class_weight': 'balanced', 'max_iter': 395, 'C': 1.0937699794981433}
# Best score: 0.71263888533865

In [ ]:
# model = LGBM(**study.best_trial.params)
# model.fit(X_train, y_train)
# preds = model.predict(X_test)
# score = macro_f1_score(y_test, preds)

In [ ]:
df_trials = study.trials_dataframe()

In [ ]:
df_trials.to_csv("SVD_LogReg_Hyperparams.csv", index=False)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['n_components',
                            'penalty',
                            'solver', 'class_weight', 'C'])

In [ ]:
def create_model(trial):
    params = {
        'C': trial.suggest_float('C', 0.1, 10, log=True),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        'gamma': trial.suggest_float('gamma', 0.1, 1, log=True),

        'tol': trial.suggest_float('tol', 1e-4, 0.1, log=True),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'cache_size': 500,

        'verbose': 100,
        'random_state': 100,
    }
        
    model = SVC(**params)
    
    return model

In [ ]:
def objective(trial, X_train=X_final_train, y_train=y_train,X_valid=X_final_valid, y_valid=y_valid,model=create_model):
#     svd_params = {
#         'n_components': trial.suggest_int('n_components', 50, 3500, step=50),
#         'algorithm': 'randomized', #trial.suggest_categorical('algorithm', ['arpack', 'randomized']),
#         'random_state': 100,
#         'n_iter': 5
#     }

#     svd = TruncatedSVD(**svd_params)
#     svd.fit(X_train)
#     X_train_svd = svd.transform(X_train)
    
    model = create_model(trial) 
#     model.fit(X_train_svd, y_train)
#     X_valid_svd = svd.transform(X_valid)
#     preds = model.predict(X_valid_svd)

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = macro_f1_score(y_valid, preds)
    
    return score

In [ ]:
# def objective(trial, X_train=X_train, y_train=y_train,X_valid=X_valid, y_valid=y_valid,model=create_model):
#     model = create_model(trial)
#     model.fit(X_train, y_train)
    
#     preds = model.predict(X_valid)
#     score = macro_f1_score(y_valid, preds)
    
#     return score

In [ ]:
%%time
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize') # can define seed for sampler
study.optimize(objective, n_trials=60, timeout=10000)

In [ ]:
print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial)
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_trial.value)  # Show the best value.
# Number of finished trials: 60
# Best parameters: {'n_components': 2350, 'C': 2.623586493072655, 'kernel': 'rbf', 'gamma': 0.692198847116744, 'tol': 0.0002174837787410716, 'class_weight': None}
# Best score: 0.7198534443262943

In [ ]:
svd = TruncatedSVD(n_components=2350, algorithm="randomized", n_iter=5, random_state=100)
svd.fit(X_final_train)
X_train_svd = svd.transform(X_final_train)
    
model = SVC(C=2.6, kernel='rbf', gamma=0.7, tol=0.0002, class_weight=None, cache_size=500, random_state=100) 
model.fit(X_train_svd, y_train)
    
X_valid_svd = svd.transform(X_final_valid)
preds = model.predict(X_valid_svd)
score = macro_f1_score(y_valid, preds)

In [ ]:
# X_test_svd = svd.transform(X_test)
# results = model.predict(X_test)
# results

In [ ]:
# idsindf = helper.to_frame()
# resultsindf = pd.DataFrame(results)
# print(resultsindf.shape, idsindf.shape)
# df_results = pd.concat([idsindf,resultsindf], axis =1)
# df_results = df_results.rename(columns={0: 'label'})
# display(df_results)

In [ ]:
# df_results.to_csv("SVD_SVM_Predictions.csv", index=False)

In [ ]:
df_trials = study.trials_dataframe()

In [ ]:
df_trials.to_csv("SVD_SVC_Hyperparams.csv", index=False)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['n_components',
                            'kernel',
                            'C',
                            'gamma', 'class_weight'])